# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
import sys
sys.path.append('../')
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
# Display sample data
city_data_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Camana,-16.6228,-72.7111,75.15,74,12,8.01,PE,few clouds
1,1,Fortuna,40.5982,-124.1573,43.14,93,0,6.91,US,clear sky
2,2,Punta Arenas,-53.1500,-70.9167,62.71,36,0,20.71,CL,clear sky
3,3,Aklavik,68.2191,-135.0107,-29.20,68,20,3.44,CA,few clouds
4,4,Pacific Grove,36.6177,-121.9166,54.07,79,0,4.00,US,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [5]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [10]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Camana,-16.6228,-72.7111,75.15,74,12,8.01,PE,few clouds
6,6,Avarua,-21.2078,-159.7750,78.85,78,40,9.22,CK,scattered clouds
8,8,Carnarvon,-24.8667,113.6333,72.91,84,0,7.72,AU,clear sky
10,10,Kapaa,22.0752,-159.3190,71.92,86,75,9.22,US,light rain
16,16,Cabo San Lucas,22.8909,-109.9124,77.50,60,52,4.00,MX,broken clouds
20,20,Puerto Ayora,-0.7393,-90.3518,82.36,87,63,7.00,EC,broken clouds
25,25,Sao Joao Da Barra,-21.6403,-41.0511,73.49,90,100,18.12,BR,light rain
26,26,Ribeira Brava,32.6500,-17.0667,72.01,95,100,10.51,PT,light rain
27,27,Kavieng,-2.5744,150.7967,80.01,80,100,4.54,PG,moderate rain
29,29,Nouadhibou,20.9310,-17.0347,78.78,47,0,4.61,MR,clear sky


In [15]:
# There aren't any null values in the rows

preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [16]:
print(len(preferred_cities_df))

243


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [17]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(5)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Camana,-16.6228,-72.7111,75.15,74,12,8.01,PE,few clouds
6,6,Avarua,-21.2078,-159.7750,78.85,78,40,9.22,CK,scattered clouds
8,8,Carnarvon,-24.8667,113.6333,72.91,84,0,7.72,AU,clear sky
10,10,Kapaa,22.0752,-159.3190,71.92,86,75,9.22,US,light rain
16,16,Cabo San Lucas,22.8909,-109.9124,77.50,60,52,4.00,MX,broken clouds


In [18]:
print(len(clean_travel_cities))

243


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Max Temp,Lat,Lng
0,Camana,PE,75.15,-16.6228,-72.7111
6,Avarua,CK,78.85,-21.2078,-159.7750
8,Carnarvon,AU,72.91,-24.8667,113.6333
10,Kapaa,US,71.92,22.0752,-159.3190
16,Cabo San Lucas,MX,77.50,22.8909,-109.9124
...,...,...,...,...,...
672,Muheza,TZ,74.23,-5.1667,38.7833
674,Ugoofaaru,MV,80.71,5.6667,73.0000
675,Caraballeda,VE,88.38,10.6112,-66.8522
678,Nortelandia,BR,79.99,-14.4547,-56.8028


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [22]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Camana,PE,75.15,-16.6228,-72.7111,
6,Avarua,CK,78.85,-21.2078,-159.7750,
8,Carnarvon,AU,72.91,-24.8667,113.6333,
10,Kapaa,US,71.92,22.0752,-159.3190,
16,Cabo San Lucas,MX,77.50,22.8909,-109.9124,
20,Puerto Ayora,EC,82.36,-0.7393,-90.3518,
25,Sao Joao Da Barra,BR,73.49,-21.6403,-41.0511,
26,Ribeira Brava,PT,72.01,32.6500,-17.0667,
27,Kavieng,PG,80.01,-2.5744,150.7967,
29,Nouadhibou,MR,78.78,20.9310,-17.0347,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [23]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [24]:
# Iterate through the hotel DataFrame

    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Camana - nearest hotel: Caja Arequipa
Avarua - nearest hotel: Paradise Inn
Carnarvon - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Cabo San Lucas - nearest hotel: Comfort Rooms
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Sao Joao Da Barra - nearest hotel: Pousada Mediterrâneo
Ribeira Brava - nearest hotel: Cheerfulway Bravamar Hotel
Kavieng - nearest hotel: Kavieng Hotel
Nouadhibou - nearest hotel: Hotel valencia
Georgetown - nearest hotel: Page 63 hostel
Bosobolo - nearest hotel: No hotel found
Jaque - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Pangkalanbuun - nearest hotel: Hotel Tiara
Santa Cruz - nearest hotel: Aviador Hotel Boutique
Galesong - nearest hotel: No hotel found
Mogadishu - nearest hotel: SYL Hotel
Noumea - nearest hotel: Hotel Gondwana
Vaini - nearest hotel: Keleti International Resort
Chuy - nearest hotel: Alerces
Salinopolis - nearest hotel: Hotel Salinópolis
Axim -

Taoudenni - nearest hotel: No hotel found
Alta Floresta - nearest hotel: Villas Hotel
Bajil - nearest hotel: فندق و أجنحة الشرق الأوسط
Tamandare - nearest hotel: Pousada Baia dos Corais
Amapa - nearest hotel: No hotel found
Boa Vista - nearest hotel: Aipana Plaza Hotel
Ouadda - nearest hotel: No hotel found
Tangua - nearest hotel: No hotel found
San Cristobal - nearest hotel: Terra Sur
Sinnamary - nearest hotel: Hôtel du fleuve
Tanete - nearest hotel: Hotel grand Sidny
Tocopilla - nearest hotel: hotel Isidora
Santa Rosa - nearest hotel: JARS RESORT
The Valley - nearest hotel: CeBlue
Varkkallai - nearest hotel: The Gateway Hotel
Maceio - nearest hotel: Maceió Hostel e Pousada
Tiruchchendur - nearest hotel: No hotel found
Hervey Bay - nearest hotel: Hervey Bay Hotel
Sao Paulo De Olivenca - nearest hotel: No hotel found
Phan Thiet - nearest hotel: Khách sạn Cà Ty
Sola - nearest hotel: No hotel found
Patos De Minas - nearest hotel: Gálatas Central Hotel
Kavaratti - nearest hotel: No hotel 

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Camana,PE,75.15,-16.6228,-72.7111,Caja Arequipa
6,Avarua,CK,78.85,-21.2078,-159.7750,Paradise Inn
8,Carnarvon,AU,72.91,-24.8667,113.6333,No hotel found
10,Kapaa,US,71.92,22.0752,-159.3190,Pono Kai Resort
16,Cabo San Lucas,MX,77.50,22.8909,-109.9124,Comfort Rooms
...,...,...,...,...,...,...
672,Muheza,TZ,74.23,-5.1667,38.7833,No hotel found
674,Ugoofaaru,MV,80.71,5.6667,73.0000,No hotel found
675,Caraballeda,VE,88.38,10.6112,-66.8522,Hotel Belmar
678,Nortelandia,BR,79.99,-14.4547,-56.8028,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [33]:
# Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == "No hotel found"].index, inplace = True)
hotel_df
clean_hotel_df = hotel_df

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Camana,PE,75.15,-16.6228,-72.7111,Caja Arequipa
6,Avarua,CK,78.85,-21.2078,-159.7750,Paradise Inn
10,Kapaa,US,71.92,22.0752,-159.3190,Pono Kai Resort
16,Cabo San Lucas,MX,77.50,22.8909,-109.9124,Comfort Rooms
20,Puerto Ayora,EC,82.36,-0.7393,-90.3518,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
664,Kisangani,CD,71.08,0.5167,25.2000,Le Chalet
667,Tupaciguara,BR,72.64,-18.5922,-48.7050,Macauba
670,Gabu,GW,84.33,12.2833,-14.2167,HOTEL VISIOM
675,Caraballeda,VE,88.38,10.6112,-66.8522,Hotel Belmar


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [37]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [43]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Max Temp", "Hotel Name", "Country","Current Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)